# 準備

## Google Colab 用の処理

下記を実行します
- ドライブのマウント
- ノートブックファイルと同じフォルダへの移動 

Googleドライブのマイドライブ を基準に DNN_code/DNN_code_colab_day4 フォルダを置くことを仮定しています。必要に応じて，パスを変更してください．

In [17]:
# Google Colab での実行かを調べる
import sys
import os
ENV_COLAB = True  if 'google.colab' in sys.modules else False 

# google drive のマウント
if ENV_COLAB:
  from google.colab import drive 
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DCGAN による画像生成

In [32]:
from PIL import Image
import numpy as np
import os, sys
 
input_dir = '/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/data_batch_1'
output_dir = '/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1'
label_file = '/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/batches.meta'
 
def read_bin(path):
  import pickle
  with open(path, 'rb') as f:
    r = pickle.load(f, encoding='bytes')
  return r
 
# 保存先ディレクトリの作成とカテゴリディレクトリの作成
try:
  os.makedirs(output_dir)
except:
  pass
labels = read_bin(label_file)
if b"label_names" in labels:
  labels = read_bin(label_file)[b"label_names"]
else:
  labels = read_bin(label_file)[b"coarse_label_names"]
for i in labels:
  try:
    dir=os.path.join(output_dir, i.decode('utf-8'))
    os.makedirs(dir)
  except:
    pass
 
# 画像バイナリの読み込み
images = read_bin(input_dir)
files = images[b"filenames"]
if b"labels" in images:
  image_labels = images[b"labels"]
else:
  image_labels = images[b"coarse_labels"]
image_data = images[b"data"]
 
# 画像ファイルとして保存
for i,_ in enumerate(files):
  file = files[i]
  image_label = image_labels[i]
  image = image_data[i]
  path = os.path.join(os.path.join(output_dir, labels[image_label].decode('utf-8'), file.decode('utf-8')))
  out_image_1 = np.reshape(image, [3, 32, 32]).transpose(1,2,0)
  out_image_2 = Image.fromarray(out_image_1)
  with open(path, mode='wb') as o:
    out_image_2.save(o)
  print("saved: {}".format(path))

ValueError: ignored

In [26]:
def convert_img_to_npy(
    img_dir="/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1/automobile", cond="*.png", save_path="/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1/automobile_np"
    ):
    """画像ファイル群を学習データ用にnumpy形式に変換する関数
    学習データが既にあれば実行する必要無し

    Args:
        img_dir (str): 画像ファイルがあるフォルダ. Defaults to ""/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1/automobile".
        cond (str): 変換する画像の条件. Defaults to "*.png".
        save_path (str): 変換後の保存先. Defaults to "/content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1/automobile_np".
    """
    dataset = []
    img_path = str(Path(img_dir) / cond)
    print("Loading Image Data from " + img_dir)
    for idx, img_path in enumerate(glob.glob(img_path)):
        # (height,width,channels)
        img_data = Image.open(img_path)
        print(str(idx) + " data loaded")
        # (1,height,width,channels)
        img_data = np.expand_dims(img_data, axis=0)
        if idx == 0:
            dataset = img_data
        else:
            dataset = np.concatenate([dataset, img_data], axis=0)
        
    dataset = (dataset - 127.5) / 127.5
    np.save(save_path, dataset)
    print("Train Data Saved")

convert_img_to_npy()

Loading Image Data from /content/drive/MyDrive/RabbitChallenge/深層学習Day4/codes/4_10_code_dcgan/cifar-10-batches-py/cifar-10-images-1/automobile
0 data loaded
1 data loaded
2 data loaded
3 data loaded
4 data loaded
5 data loaded
6 data loaded
7 data loaded
8 data loaded
9 data loaded
10 data loaded
11 data loaded
12 data loaded
13 data loaded
14 data loaded
15 data loaded
16 data loaded
17 data loaded
18 data loaded
19 data loaded
20 data loaded
21 data loaded
22 data loaded
23 data loaded
24 data loaded
25 data loaded
26 data loaded
27 data loaded
28 data loaded
29 data loaded
30 data loaded
31 data loaded
32 data loaded
33 data loaded
34 data loaded
35 data loaded
36 data loaded
37 data loaded
38 data loaded
39 data loaded
40 data loaded
41 data loaded
42 data loaded
43 data loaded
44 data loaded
45 data loaded
46 data loaded
47 data loaded
48 data loaded
49 data loaded
50 data loaded
51 data loaded
52 data loaded
53 data loaded
54 data loaded
55 data loaded
56 data loaded
57 data load

In [27]:
args_dict = {
    'exp_name': "exp_2", # 実験名(この名前のフォルダが./expに生成される)
    "noize_dim": 100, # Gの入力乱数ベクトルの次元
    "batch_size": 128, # バッチサイズ
    "num_epoch": 100, # 学習エポック数
    "make_logs": True, # D,GのLossをLogとして記録するかどうか
    "restore_ckpt": False, # 学習済みモデルのckptを読み込むかどうか
    "sample_freq": 10, # 学習中の画像生成頻度(エポック数)
    "ckpt_freq": 10, # 学習中のckpt保存頻度(エポック数)
    "adam": {"lr":2e-4, # 学習率
            "beta_1":0.5,
            "beta_2":0.9,
            "epsilon":1e-8}
    }

In [28]:
train_data_path = "./cifar-10-batches-py/cifar-10-images-1/automobile_np.npy"
# 学習データサイズ(data_num, H:64, W:64, RGB:3), 値の範囲は[-1,1]
print("Loading Train Data")
train_data = np.load(train_data_path)
print("Train Data Loaded")

Loading Train Data
Train Data Loaded


In [29]:
model = DCGAN(args_dict)

========================== Model Initialization Done ===========================


In [31]:
model.train(train_data)

================================ Training Start ================================


ValueError: ignored

In [ ]:
# 学習済みモデルを用いて画像生成
model.generate_img()